# Вступительная часть

В этом пет-проекте я хочу создать файлы (excel и word) с экономическим анализом отчётности компании ГК "Агроэко". Выбор пал на эту компанию, поскольку в студенческую бытность я проходила в этой компании стажировку и получила необходимые документы для курсовых работ.

Аналитика будет производится по трём направлениям:
- Анализ финансовой устойчивости,
- Анализ ликвидности предприятия,
- Анализ текущей кредитоспособности.

В результате данного пет проекта планируется получить:
1. код, который будет автоматизировать анализ отчётности предприятия (с намёком на масштабирование)
2. файлы с аналитикой.

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np

# Для записи в excel
import openpyxl as opx
from openpyxl import Workbook
from openpyxl.utils.cell import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows

# Запись в word
!pip install python-docx
from docx import Document
from docx.enum.section import WD_ORIENT                       # для смены ориентации листа
from docx.shared import Mm                                    # для установления размеров листа
from docx.enum.text import WD_COLOR_INDEX                     # для выделения текста
from docx.shared import Pt                                    # для установления размера текста

In [ ]:
# Парсинг отчётов и объединение отчётов в один датафрейм
balance = pd.read_excel('/content/ГК Агроэко.xlsx', sheet_name='бух.бал.', header=2,
                        index_col='Код показателя', usecols=[1,2,3,4,5])
income_statement = pd.read_excel('/content/ГК Агроэко.xlsx', sheet_name='фин.результаты',
                                 header=2, index_col='Код показателя', usecols=[1,2,3,4,5])

statements = pd.concat([balance, income_statement])
statements.head()

,Показатель,2018-12-31 00:00:00,2017-12-31 00:00:00,2016-12-31 00:00:00
Код показателя,,,,
NaN,Актив,NaN,NaN,NaN
NaN,I. ВНЕОБОРОТНЫЕ АКТИВЫ,0.0,NaN,NaN
1110.0,Нематериальные активы,0.0,0.0,0.0
1120.0,Результаты исследований и разработок,0.0,0.0,0.0
1130.0,Нематериальные поисковые активы,0.0,0.0,0.0


In [ ]:
# Создание нового excel файла
wb = Workbook()

# Создание docx документа
doc = Document()

# Экономический анализ

## Финансовая устойчивость

Рассчитаю 10 коэффициентов финансовой устойчивости.

In [ ]:
def fin_stab_calc(df):

  fin_stab_df = pd.DataFrame()

  fin_stab_df['Коэффициент автономии'] = [df.loc[1300, col] / df.loc[1700, col] if df.loc[1700, col]!=0 else '-' for col in df.iloc[:,1:].columns]
  fin_stab_df['Коэффициент финансового левериджа'] = [(df.loc[1400, col] + df.loc[1500, col]) / df.loc[1300, col] if df.loc[1300, col]!=0 else '-'
                                                      for col in df.iloc[:,1:].columns]
  fin_stab_df['Коэффициент обеспеченности собственными оборотными средствами'] = [(df.loc[1200, col] + df.loc[1500, col]) / df.loc[1200, col]
                                                                                  if df.loc[1200, col]!=0 else '-' for col in df.iloc[:,1:].columns]
  fin_stab_df['Индекс постоянного актива'] = [df.loc[1100, col] / df.loc[1300, col] if df.loc[1300, col] !=0 else '-' for col in df.iloc[:,1:].columns]
  fin_stab_df['Коэффициент покрытия инвестиций (финансовой устойчивости)'] = [(df.loc[1300, col] + df.loc[1400, col])/ df.loc[1600, col]
                                                                              if df.loc[1600, col]!=0 else '-' for col in df.iloc[:,1:].columns]
  fin_stab_df['Коэффициент маневренности собственного капитала'] = [(df.loc[1300, col] - df.loc[1100, col])/ df.loc[1300, col] if df.loc[1300, col] !=0 else '-'
                                                                    for col in df.iloc[:,1:].columns]
  fin_stab_df['Коэффициент мобильности имущества'] = [df.loc[1200, col] / df.loc[1700, col] if df.loc[1700, col]!=0 else '-' for col in df.iloc[:,1:].columns]
  fin_stab_df['Коэффициент мобильности оборотных средств'] = [(df.loc[1240, col] + df.loc[1250, col])/ df.loc[1200, col] if df.loc[1200, col]!=0 else '-'
                                                              for col in df.iloc[:,1:].columns]
  fin_stab_df['Коэффициент обеспеченности запасов'] = [(df.loc[1300, col] - df.loc[1100, col])/ df.loc[1210, col] if df.loc[1210, col] !=0 else '-'
                                                       for col in df.iloc[:,1:].columns]
  fin_stab_df['Коэффициент краткосрочной задолженности'] = [df.loc[1500, col] / (df.loc[1400, col] + df.loc[1500, col]) if  (df.loc[1400, col] + df.loc[1500, col])!=0 else '-'
                                                            for col in df.iloc[:,1:].columns]

  return fin_stab_df


Поскольку анализ финансовой устойчивости и ликвидности по структуре будут одинаковыми, напишу функцию, которая будет переносить в excel и форматировать таблицы.

In [ ]:
def transfer_table(ws, df, titles):

  # Перенос таблицы в excel
  _rows = dataframe_to_rows(df)
  _row = len(tuple(ws.rows))

  for _r_idx, _row_ in enumerate(_rows, 1):

    if None not in _row_:

      # Перенос текста в ячейке
      ws.cell(row=1+_r_idx, column=2).alignment = opx.styles.Alignment(wrapText=True)

      # Перенос значения из таблицы в excel
      for _c_idx, _value in enumerate(_row_, 1):
        ws.cell(row=1 +_r_idx, column=1+_c_idx, value=_value)
        ws.cell(row=1 +_r_idx, column=1+_c_idx).number_format = '0.000'
        ws.column_dimensions[get_column_letter(1+_c_idx)].width = 15

  # Заголовки таблицы
  for _c in range(2,3+len(statements.iloc[:, 1:].columns)):
    ws.cell(row=_row+3, column=_c, value=titles[_c-2])
    ws.cell(row=_row+3, column=_c).font = opx.styles.Font(bold=True, size=12)
    ws.cell(row=_row+3, column=_c).border = opx.styles.Border(bottom=opx.styles.Side(border_style='medium'))

    if _c!=2:
      ws.cell(row=_row+3, column=_c).number_format = 'd.m.yyyy'

  # Установление ширины столбца
  ws.column_dimensions['B'].width = 45

### excel

In [ ]:
ws = wb[wb.sheetnames[0]]
ws.title = 'Анализ финансовой устойчивости'
ws.sheet_view.showGridLines = False

# Перенос таблицы в excel файл
temp_1 = fin_stab_calc(statements)
transfer_table(ws, temp_1.T, ['Показатель'] + statements.iloc[:, 1:].columns.tolist())

Буду обозначать цветом те значения коэффициентов, которые не попадают в свои нормативные границы.

Также в расчёт коэффициентов я внесла правило, согласно которому знаменатель не может быть равен нулю. В бухгалтерской отчётности не все строки имеют значение больше нуля. Это может быть связано как и с особенностями вида деятельности предприятия, с отсутствием какого-либо движения по счёту, или с тем фактом,что обороты по счёту в итоге просто вышли в ноль.

Такое правило привело к тому, что расчёт некоторых коэффициентов даёт строковое значение '-', у которого нет "границ адекватности". Поэтому в коде дополнительно прописана конструкция try - except.

In [ ]:
# Обозначение границ адекватности
for _c in range(3,3+len(statements.iloc[:, 1:].columns)):

  try:
    if ws.cell(row=4, column=_c).value > 0.55 and ws.cell(row=4, column=_c).value < 0.75:
      ws.cell(row=4, column=_c).fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')
  except:
    print(f'Проблема в колонке №{_c} строке 4')

  try:
    if ws.cell(row=5, column=_c).value > 0.33 and ws.cell(row=5, column=_c).value < 0.82:
      ws.cell(row=5, column=_c).fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')
  except:
    print(f'Проблема в колонке №{_c} строке 5')

  try:
    if ws.cell(row=6, column=_c).value > 0.1:
      ws.cell(row=6, column=_c).fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')
  except:
    print(f'Проблема в колонке №{_c} строке 6')

  try:
    if ws.cell(row=8, column=_c).value > 0.85:
      ws.cell(row=8, column=_c).fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')
  except:
    print(f'Проблема в колонке №{_c} строке 8')

  try:
    if ws.cell(row=9, column=_c).value > 0.05:
      ws.cell(row=9, column=_c).fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')
  except:
    print(f'Проблема в колонке №{_c} строке 9')

  try:
    if ws.cell(row=12, column=_c).value > 0.5:
      ws.cell(row=12, column=_c).fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')
  except:
    print(f'Проблема в колонке №{_c} строке 12')

Проблема в колонке №3 строке 12
Проблема в колонке №5 строке 12


### word

In [ ]:
def transfer_table_to_docx(df):
  # Инициализирцю таблицу
  t = doc.add_table(rows=df.shape[0]+1, cols=df.shape[1])

  # Add the column headings
  for j in range(df.shape[1]):
      t.cell(0, j).text = df.columns[j]

  # add the rest of the data frame
  for i in range(df.shape[0]):
      for j in range(df.shape[-1]):
        try:
          t.cell(i+1,j).text = str(np.round(df.values[i,j],3))
        except:
          t.cell(i+1,j).text = str(df.values[i,j])


In [ ]:
# Меняю ориентацию листа на альбомную
for section in doc.sections:
  section.orientation = WD_ORIENT.LANDSCAPE
  section.page_width = Mm(297)
  section.page_height = Mm(210)


head = doc.add_heading('Финансовая устойчивость', 1)

doc.add_paragraph("Финансовая устойчивость характеризует степень зависимости организации от внешних источников финансирования и ее " + \
                  "возможность отвечать по своим текущим обязательствам в будущем. Таким образом, показатели устойчивости призваны отражать " +\
                  "степень надежности компании в перспективе.")

# Разобью первую таблицу на две части
for i in range(1,3):
  doc.add_paragraph(f'Таблица #{i}.')
  if i==1:
    transfer_table_to_docx(pd.concat([pd.DataFrame(statements.iloc[:, 1:].columns.tolist(), columns=['Год']), temp_1.iloc[:,:5]], axis=1))
  else:
    transfer_table_to_docx(pd.concat([pd.DataFrame(statements.iloc[:, 1:].columns.tolist(), columns=['Год']), temp_1.iloc[:,5:]], axis=1))
  doc.add_page_break()


## Ликвидность

Рассчитаю коэффициенты финансовой ликвидности.

In [ ]:
def liq_calx(df):
  liq_df = pd.DataFrame()

  liq_df['Коэффициент общей ликвидности'] = [df.loc[1200, col] / df.loc[1500, col] if df.loc[1500, col]!=0 else '-' for col in df.iloc[:,1:].columns]
  liq_df['Коэффициент быстрой (промежуточной) ликвидности'] = [(df.loc[1230, col] + df.loc[1240, col] + df.loc[1250, col] + df.loc[1260, col])/
                                                               df.loc[1500, col] if df.loc[1500, col]!=0 else '-' for col in df.iloc[:,1:].columns]
  liq_df['Коэффициент абсолютной ликвидности'] = [(df.loc[1240, col] + df.loc[1250, col])/
                                                               df.loc[1500, col] if df.loc[1500, col]!=0 else '-' for col in df.iloc[:,1:].columns]
  liq_df['Коэффициент текущей ликвидности'] = [(df.loc[1200, col] - df.loc[1230, col]) / df.loc[1500, col]
                                               if df.loc[1500, col]!=0 else '-' for col in df.iloc[:,1:].columns]

  return liq_df

### excel

In [ ]:
# Инициация нового листа
ws = wb.create_sheet("Анализ ликвидности")

# Расчёт коэффициентов ликвидности, перенос таблицы в новый лист excel отчёта
temp_2 = liq_calx(statements)
transfer_table(ws, temp_2.T, ['Показатель'] + statements.iloc[:, 1:].columns.tolist())

In [ ]:
temp_2

,Коэффициент общей ликвидности,Коэффициент быстрой (промежуточной) ликвидности,Коэффициент абсолютной ликвидности,Коэффициент текущей ликвидности
0,0.032306,0.002474,0.000000,0.029832
1,0.000478,0.000223,0.000000,0.000279
2,0.019349,0.019349,0.019311,0.019349


In [ ]:
# Создам новый столбец, в котором будут выводы о ликвидности рассматриваемого предприятия

if ws.cell(row=4, column=3).value < 2:
  ws.cell(row=4, column=6, value='У предприятия недостаточно средств для погашения краткосрочных обязательств в долгосрочном периоде.').fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='00FF8080')
else:
  ws.cell(row=4, column=6, value='У предприятия достаточно средств для погашения краткосрочных обязательств в долгосрочном периоде.').fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')



if ws.cell(row=5, column=3).value < 1:
  ws.cell(row=5, column=6, value='Предприятие не способно погашать свои краткосрочные обязательства за счёт продажи быстро и средне ликвидных активов.').fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='00FF8080')
else:
  ws.cell(row=5, column=6, value='Предприятие не способно погашать свои краткосрочные обязательства за счёт продажи быстро и средне ликвидных активов.').fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')



if ws.cell(row=6, column=3).value < 0.2 or ws.cell(row=6, column=3).value > 0.4:
  ws.cell(row=6, column=6, value='Низкая платёжеспособность предприятия').fill  = opx.styles.fills.PatternFill(patternType='solid', fgColor='00FF8080')
else:
  ws.cell(row=6, column=6, value='Умеренная платёжеспособность предприятия').fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')



if ws.cell(row=7, column=3).value < 2:
  ws.cell(row=7, column=6, value='Низкая ликвидность активов компании.').fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='00FF8080')
else:
  ws.cell(row=7, column=6, value='Высокая ликвидность активов компании.').fill  = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')


# Заголовок столбца с выводами
ws.cell(row=3, column=6, value='Вывод')
ws.cell(row=3, column=6).font = opx.styles.Font(bold=True, size=12)
ws.cell(row=3, column=6).border = opx.styles.Border(bottom=opx.styles.Side(border_style='medium'))


In [ ]:
# Украшательства
ws.sheet_view.showGridLines = False

for _r in range(4, 8):
  ws.cell(row=_r, column=6).alignment = opx.styles.Alignment(wrapText=True)

ws.column_dimensions['F'].width = 50

### word

In [ ]:
head = doc.add_heading('Ликвидность', 1)

doc.add_paragraph("Показатели ликвидности характеризуют платежеспособность компании. При этом рассчитывается только " + \
                  "возможность погашать свои краткосрочные обязательства.")
doc.add_paragraph('\n')

# Перенос второй таблиц в word
transfer_table_to_docx(pd.concat([pd.DataFrame(statements.iloc[:, 1:].columns.tolist(), columns=['Год']), temp_2], axis=1))
doc.add_paragraph('\n')
doc.add_page_break()


## Кредитоспособность

Кредитоспособность буду оценивать в соответствии с методикой Сбербанка. По этой методологии заёмщики делятся в зависимости от полученной суммы баллов на три класса:
- первоклассные 	– кредитование которых не вызывает сомнений (сумма баллов до 1,25);
- второго класса	 – кредитование требует взвешенного подхода (свыше 1,25 но меньше 2,35);
- третьего класса 	– кредитование связано с повышенным риском (2,35 и выше).


In [ ]:
def credit_calc(df):
  credit_df = pd.DataFrame()

  credit_df['Коэффициент наличия собственных средств'] = [(df.loc[1300, col] - df.loc[1100, col])/ df.loc[1200, col]
                                                       if df.loc[1200, col]!=0 else '-' for col in df.iloc[:,1:].columns]

  credit_df['Рентабельность продукции'] = [df.loc[2300, col] / df.loc[2120, col]
                                        if df.loc[2120, col]!=0 else '-' for col in df.iloc[:,1:].columns]

  credit_df['Рентабельность деятельности предприятия'] = [df.loc[2400, col] / df.loc[2120, col]
                                                       if df.loc[2110, col]!=0 else '-' for col in df.iloc[:,1:].columns]

  return credit_df

In [ ]:
col_1 = ['Показатель',
        'Коэффициент абсолютной ликвидности',
        'Коэффициент промежуточной (быстрой) ликвидности',
        'Коэффициент текущей ликвидности',
        'Коэффициент наличия собственных средств (кроме торговых и лизинговых организаций)',
        'Рентабельность продукции',
        'Рентабельность деятельности предприятия']

col_2 = ['Фактическое значение',
         *[liq_calx(statements).T.iloc[i,0] for i in [3,2,-1]],
         *credit_calc(statements).T.iloc[:,0].values.tolist()]


bounds = [[0.05,0.1], [0.5, 0.8], [1, 1.5], [0.25, 0.4], [0.1, 0.1], [0.06, 0.06]]
col_3 = ['Категория']
for i in range(0, len(col_2[1:])):
  try:
    if i!=4 and i!=5:
      col_3.append(3 if col_2[1:][i] < bounds[i][0] else (1 if col_2[1:][i] > bounds[i][1] else 2))
    else:
      col_3.append(2 if col_2[1:][i] < bounds[i][0] else 1)
  except:
    col_3.append('-')


col_4 = ['Вес показателя', 0.05, 0.1, 0.4, 0.2, 0.15, 0.1]

col_5 = ['Расчет баллов']
for i in range(1,7):
  try:
    col_5.append(col_3[i]*col_4[i])
  except:
    col_5.append(0)


In [ ]:
credit_calc(statements)

,Коэффициент наличия собственных средств,Рентабельность продукции,Рентабельность деятельности предприятия
0,-29.954048,-,-
1,-6889.733108,-,-
2,-100.134216,-,-


### excel

In [ ]:
# Инициация нового листа
ws = wb.create_sheet("Кредитоспособность")

cols = [col_1, col_2, col_3, col_4, col_5]

for _r in range(3, len(col_1)+3):
  for _c in range(0, len(cols)):
    ws.cell(row=_r, column=_c+2, value=cols[_c][_r-3])
    ws.cell(row=_r, column=_c+2).number_format = '0.000'

    ws.cell(row=3, column=_c+2).border = opx.styles.Border(bottom=opx.styles.Side(border_style='medium'))
    ws.cell(row=3, column=_c+2).font = opx.styles.Font(bold=True, size=12)


# Итоговый расчёт баллов
ws.cell(row=10, column=6, value=np.round(sum(col_5[1:]),3))

# Вывод класса заёмщика
if ws.cell(row=10, column=6).value > 2.35:
  ws.cell(row=11, column=6, value='Кредитование связано с повышенным риском.')
  ws.cell(row=10, column=6).fill = opx.styles.fills.PatternFill(patternType='solid', fgColor='00FF8080')
elif ws.cell(row=10, column=6).value < 1.25:
  ws.cell(row=11, column=6, value='Кредитование предприятия не вызывает сомнений.')
  ws.cell(row=10, column=6).fill  = opx.styles.fills.PatternFill(patternType='solid', fgColor='0099CC00')
else:
  ws.cell(row=11, column=6, value='Кредитование требует взвешенного подхода.')
  ws.cell(row=10, column=6).fill  = opx.styles.fills.PatternFill(patternType='solid', fgColor='00FFFF99')


In [ ]:
# Украшательства
ws.sheet_view.showGridLines = False

for _c in range(2, 7):
  ws.cell(row=3, column=_c).alignment = opx.styles.Alignment(wrapText=True, horizontal='center')
  ws.cell(row=_c+1, column=2).alignment = opx.styles.Alignment(wrapText=True)


ws.column_dimensions['B'].width = 40
for _c in ['C', 'D', 'E', 'F']:
  ws.column_dimensions[_c].width = 20


### word

In [ ]:
temp_3 = pd.DataFrame([col_1, col_2, col_3, col_4, col_5])
temp_3.columns = temp_3.iloc[0]
temp_3.drop(index=0, inplace=True)


In [ ]:
head = doc.add_heading('Кредитоспособность', 1)
doc.add_paragraph("Кредитоспособность — характеристика заемщика, характеризующая возможность заёмщика воврмя погасить кредит.")
doc.add_paragraph('\n')

# Перенос третьей таблиц в word
transfer_table_to_docx(temp_3)

# Вывод класса заёмщика
if sum(col_5[1:]) > 2.35:
  p = doc.add_paragraph()
  run = p.add_run('Кредитование связано с повышенным риском.')
  run.font.highlight_color = WD_COLOR_INDEX.RED
elif sum(col_5[1:]) < 1.25:
  p = doc.add_paragraph()
  run = p.add_run('Кредитование предприятия не вызывает сомнений.')
  run.font.highlight_color = WD_COLOR_INDEX.GREEN
else:
  p = doc.add_paragraph()
  run = p.add_run('Кредитование требует взвешенного подхода.')
  run.font.highlight_color = WD_COLOR_INDEX.YELLOW


In [ ]:
# Сохранение
wb.save('fin_statements_analysis_rus.xlsx')
doc.save('fin_statements_analysis_rus.docx')